In [3]:
import requests
import folium
from folium import Circle, PolyLine
import heapq

In [21]:
help_stations_with_phone = [
    {"coordinates": (30.2837296, 77.997745), "phone_number": "+917617641909"},
    {"coordinates": (30.304785, 78.0209032), "phone_number": "+919389969916"},
    {"coordinates": (30.2909374, 78.0498482), "phone_number": "+917617641909"},
    {"coordinates": (30.3108936, 78.0121377), "phone_number": "+919389969916"},
    {"coordinates": (30.2820388, 78.002518), "phone_number": "+917617641909"},
    {"coordinates": (30.2875988, 78.0231428), "phone_number": "+919389969916"},
    {"coordinates": (30.2789269, 78.0012372), "phone_number": "+917617641909"},
    {"coordinates": (30.3125064, 78.0219506), "phone_number": "+919389969916"},
    {"coordinates": (30.3029695, 78.0214584), "phone_number": "+917617641909"},
    {"coordinates": (30.2919587, 77.9986905), "phone_number": "+919389969916"},
    {"coordinates": (30.311503, 77.982168), "phone_number": "+917617641909"},
    {"coordinates": (30.2864273, 78.0480676), "phone_number": "+919389969916"},
    {"coordinates": (30.3070576, 78.0095762), "phone_number": "+917617641909"}
]


# --- Unload Coordinates ---
help_stations = [station["coordinates"] for station in help_stations_with_phone]
phone = []

In [22]:
user_location = (30.269745410969797, 77.99290170226186)

In [23]:

import requests
import folium
from folium import Circle, PolyLine

# --- User and Help Station Coordinates ---
user_location = (30.269745410969797, 77.99290170226186)

# --- Initialize phone list ---
phone = []

# --- Function to get distance using OSRM ---
def get_osrm_distance(origin, destination):
    url = f"http://router.project-osrm.org/route/v1/foot/{origin[1]},{origin[0]};{destination[1]},{destination[0]}"
    params = {"overview": "full", "geometries": "geojson"}
    response = requests.get(url, params=params)
    data = response.json()
    if 'routes' in data and len(data['routes']) > 0:
        distance_km = data['routes'][0]['distance'] / 1000
        coords = [(coord[1], coord[0]) for coord in data['routes'][0]['geometry']['coordinates']]
        return distance_km, coords
    else:
        return float('inf'), []

# --- Get distances from user to each help station ---
distances = []
for station in help_stations:
    dist_km, _ = get_osrm_distance(user_location, station)
    distances.append(dist_km)

# --- Get 3 Nearest Stations ---
nearest = sorted(zip(help_stations, distances), key=lambda x: x[1])[:3]

# --- Extract Phone Numbers for the Nearest Stations ---
for station, _ in nearest:
    for station_info in help_stations_with_phone:
        if station_info["coordinates"] == station:
            phone.append(station_info["phone_number"])

# --- Print distances and phone numbers of the 3 nearest help stations ---
print("\n📍 3 Nearest Help Stations (by walking distance):")
for i, (station, dist) in enumerate(nearest):
    print(f"{i+1}. Location: {station}, Distance: {dist:.2f} km, Phone: {phone[i]}")

# --- Create Map ---
m = folium.Map(location=user_location, zoom_start=15)

# User location
folium.Marker(user_location, tooltip="User", icon=folium.Icon(color='blue')).add_to(m)
Circle(user_location, radius=1500, color="blue", fill=False).add_to(m)

# All help stations (gray)
for station in help_stations:
    folium.Marker(station, tooltip="Help Station", icon=folium.Icon(color="gray")).add_to(m)

# Nearest stations (green)
for i, (station, dist) in enumerate(nearest):
    folium.Marker(
        station,
        tooltip=f"Help Station {i+1} ({dist:.2f} km), Phone: {phone[i]}",
        icon=folium.Icon(color="green")
    ).add_to(m)

    # Draw route from user to station
    _, coords = get_osrm_distance(user_location, station)
    PolyLine(coords, color='green', weight=4, opacity=0.7).add_to(m)

# Save map
m.save("live_osrm_nearest_help_stations_with_phone.html")
print("✅ Map saved as 'live_osrm_nearest_help_stations_with_phone.html'")
m


📍 3 Nearest Help Stations (by walking distance):
1. Location: (30.2789269, 78.0012372), Distance: 1.81 km, Phone: +917617641909
2. Location: (30.2820388, 78.002518), Distance: 2.13 km, Phone: +917617641909
3. Location: (30.2837296, 77.997745), Distance: 2.96 km, Phone: +917617641909
✅ Map saved as 'live_osrm_nearest_help_stations_with_phone.html'


In [14]:
def dijkstra(graph, start):
    pq = [(0, start)]
    distances = {start: 0}
    previous_nodes = {start: None}

    while pq:
        current_distance, current_node = heapq.heappop(pq)

        if current_distance > distances.get(current_node, float('inf')):
            continue

        for neighbor, weight in graph.get(current_node, {}).items():
            distance = current_distance + weight
            if distance < distances.get(neighbor, float('inf')):
                distances[neighbor] = distance
                previous_nodes[neighbor] = current_node
                heapq.heappush(pq, (distance, neighbor))

    return distances, previous_nodes

In [15]:
def get_osrm_distance(origin, destination):
    url = f"http://router.project-osrm.org/route/v1/foot/{origin[1]},{origin[0]};{destination[1]},{destination[0]}"
    params = {"overview": "full", "geometries": "geojson"}
    response = requests.get(url, params=params)
    data = response.json()
    if 'routes' in data and len(data['routes']) > 0:
        distance_km = data['routes'][0]['distance'] / 1000
        coords = [(coord[1], coord[0]) for coord in data['routes'][0]['geometry']['coordinates']]
        return distance_km, coords
    else:
        return float('inf'), []

In [16]:
graph = {}
for station in help_stations:
    graph[station] = {}
    for other_station in help_stations:
        if station != other_station:
            distance, _ = get_osrm_distance(station, other_station)
            graph[station][other_station] = distance

In [17]:
graph[user_location] = {}
for station in help_stations:
    distance, _ = get_osrm_distance(user_location, station)
    graph[user_location][station] = distance

distances, previous_nodes = dijkstra(graph, user_location)
nearest = sorted(zip(help_stations, distances.values()), key=lambda x: x[1])[:3]

for station, _ in nearest:
    for station_info in help_stations_with_phone:
        if station_info["coordinates"] == station:
            phone.append(station_info["phone_number"])

In [18]:
print("\n📍 3 Nearest Help Stations (by walking distance):")
for i, (station, dist) in enumerate(nearest):
    print(f"{i+1}. Location: {station}, Distance: {dist:.2f} km, Phone: {phone[i]}")


📍 3 Nearest Help Stations (by walking distance):
1. Location: (30.2837296, 77.997745), Distance: 0.00 km, Phone: +917617641909
2. Location: (30.3125064, 78.0219506), Distance: 1.81 km, Phone: +919389969916
3. Location: (30.2875988, 78.0231428), Distance: 2.13 km, Phone: +919389969916


In [20]:

import requests
import folium
from folium import Circle, PolyLine

# --- User and Help Station Coordinates ---
user_location = (30.3245, 78.0430)

# --- Initialize phone list ---
phone = []

# --- Function to get distance using OSRM ---
def get_osrm_distance(origin, destination):
    url = f"http://router.project-osrm.org/route/v1/foot/{origin[1]},{origin[0]};{destination[1]},{destination[0]}"
    params = {"overview": "full", "geometries": "geojson"}
    response = requests.get(url, params=params)
    data = response.json()
    if 'routes' in data and len(data['routes']) > 0:
        distance_km = data['routes'][0]['distance'] / 1000
        coords = [(coord[1], coord[0]) for coord in data['routes'][0]['geometry']['coordinates']]
        return distance_km, coords
    else:
        return float('inf'), []

# --- Get distances from user to each help station ---
distances = []
for station in help_stations:
    dist_km, _ = get_osrm_distance(user_location, station)
    distances.append(dist_km)

# --- Get 3 Nearest Stations ---
nearest = sorted(zip(help_stations, distances), key=lambda x: x[1])[:3]

# --- Extract Phone Numbers for the Nearest Stations ---
for station, _ in nearest:
    for station_info in help_stations_with_phone:
        if station_info["coordinates"] == station:
            phone.append(station_info["phone_number"])

# --- Print distances and phone numbers of the 3 nearest help stations ---
print("\n📍 3 Nearest Help Stations (by walking distance):")
for i, (station, dist) in enumerate(nearest):
    print(f"{i+1}. Location: {station}, Distance: {dist:.2f} km, Phone: {phone[i]}")

# --- Create Map ---
m = folium.Map(location=user_location, zoom_start=15)

# User location
folium.Marker(user_location, tooltip="User", icon=folium.Icon(color='blue')).add_to(m)
Circle(user_location, radius=1500, color="blue", fill=False).add_to(m)

# All help stations (gray)
for station in help_stations:
    folium.Marker(station, tooltip="Help Station", icon=folium.Icon(color="gray")).add_to(m)

# Nearest stations (green)
for i, (station, dist) in enumerate(nearest):
    folium.Marker(
        station,
        tooltip=f"Help Station {i+1} ({dist:.2f} km), Phone: {phone[i]}",
        icon=folium.Icon(color="green")
    ).add_to(m)

    # Draw route from user to station
    _, coords = get_osrm_distance(user_location, station)
    PolyLine(coords, color='green', weight=4, opacity=0.7).add_to(m)

# Save map
m.save("live_osrm_nearest_help_stations_with_phone.html")
print("✅ Map saved as 'live_osrm_nearest_help_stations_with_phone.html'")
m

✅ Map saved as 'live_osrm_nearest_help_stations_with_phone.html'


In [ ]:
print(phone)

['+917617641909', '+919389969916', '+919389969916']


In [ ]:
from twilio.rest import Client

account_sid = 'AC4c024fc964c6236c3b439abd84d49329'
auth_token = 'cfc033a413a9832a55f70659e5353adb'

from_phone = '+19137330629'
message = "Help "


client = Client(account_sid, auth_token)

for to_phone in phone:
    try:
        message_sent = client.messages.create(
            body=message,
            from_=from_phone,
            to=to_phone
        )
        print(f"Message sent to {to_phone}: {message_sent.sid}")
    except Exception as e:
        print(f"Failed to send message to {to_phone}: {e}")

Message sent to +917617641909: SM60408d8a237ede446f38a823fd96c0e1
Message sent to +919389969916: SM176dd97f5117b4950c0867efcf345ec7
Message sent to +919389969916: SM3d6010f80ae096c7c300f0ecf8184119
